In [24]:
import numpy as np

In [25]:
np.random.seed(0)

In [26]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'e': 0, 'd': 1, 'r': 2, 'l': 3, 'h': 4, 'w': 5, ' ': 6, 'o': 7},
 {0: 'e', 1: 'd', 2: 'r', 3: 'l', 4: 'h', 5: 'w', 6: ' ', 7: 'o'})

In [27]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [28]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [29]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [30]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [31]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([4, 0, 3, 3, 7, 6, 5, 7, 2, 3], [0, 3, 3, 7, 6, 5, 7, 2, 3, 1])

In [32]:
sample("h", 4)

'oww '

In [40]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t] * dh_dz1.T * dL_dh.T
        dL_dbh +=  dh_dz1 * dL_dh

        # raise RuntimeError

        dF_dh = dL_dh * (dh_dz1 @ Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 11.441951838284059
[Loss]: 2.4575615294448836
[Loss]: 0.17105415269927243
[Loss]: 0.1756911138255791
[Loss]: 4.301379457472539
[Loss]: 0.16715079366591776
[Loss]: 0.20270755962373097
[Loss]: 9.009404894917743
[Loss]: 6.1812131358699745
[Loss]: 11.901331554344003
[Loss]: 6.196250577762493
[Loss]: 2.7809300030282307
[Loss]: 0.9732807265446808
[Loss]: 2.6957815124401834
[Loss]: 5.979524001494333
[Loss]: 1.78435540451556
[Loss]: 7.16889769081539
[Loss]: 10.372924103439392
[Loss]: 6.6079802575047175
[Loss]: 2.505708161540179
[Loss]: 2.0895374333557117
[Loss]: 4.030525385314281
[Loss]: 8.04274608889471
[Loss]: 3.604678843672761
[Loss]: 0.5341681722827833
[Loss]: 2.283785096223901
[Loss]: 0.24692449453968893
[Loss]: 5.875069379570699
[Loss]: 10.19784068006303
[Loss]: 0.49795366558945087
[Loss]: 23.06719381105773
[Loss]: 0.4592649710019393
[Loss]: 0.3566240088264186
[Loss]: 0.14539909295406786
[Loss]: 2.1232443437829285
[Loss]: 7.435278321871906
[Loss]: 0.8074090360885473
[Loss]: 9.950

In [41]:
sample("h", 10)

'ello worll'